# Song recommendation final

In [92]:
import numpy as np
import pandas as pd
import spotipy as sp
import config
from time import sleep
from random import randint
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import random 
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from IPython.display import Image
from IPython.display import display

In [93]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!")

In [94]:
scaler2 = load("Model/scaler.pickle")
scaler2

C:\Users\merce\anaconda3\envs\Da_Env1\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


StandardScaler()

In [95]:
kmeans2 = load("Model/kmeans_4.pickle")
kmeans2

C:\Users\merce\anaconda3\envs\Da_Env1\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator KMeans from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


KMeans(n_clusters=20, random_state=1234)

# Loading data

In [96]:
df=pd.read_csv("Data/hot100.csv")
clus_df=pd.read_csv("Data/clustered_df.csv")
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id,
                                                           client_secret=config.client_secret))
#df1=pd.read_csv("Data/songs_db.csv")

In [97]:
#clus_df["uris"]=df1.uris

1 attach uris columns to clus_df
2 extract track uri value from recommender line
3

# Connect with Spotify

In [98]:
# Conect with Spotify in order to get the most important features of a song: 
#def spotify_search():

song = sp.search(q="Bohemian Rhapsody", limit=1) 
#pprint.pprint(song['tracks']['items'][0]['uri'])
#song["tracks"]["items"][0]["uri"]
sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

{'danceability': 0.392,
 'energy': 0.402,
 'key': 0,
 'loudness': -9.961,
 'mode': 0,
 'speechiness': 0.0536,
 'acousticness': 0.288,
 'instrumentalness': 0,
 'liveness': 0.243,
 'valence': 0.228,
 'tempo': 143.883,
 'type': 'audio_features',
 'id': '7tFiyTwD0nx5a1eklYtX2J',
 'uri': 'spotify:track:7tFiyTwD0nx5a1eklYtX2J',
 'track_href': 'https://api.spotify.com/v1/tracks/7tFiyTwD0nx5a1eklYtX2J',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7tFiyTwD0nx5a1eklYtX2J',
 'duration_ms': 354320,
 'time_signature': 4}

In [99]:
#def spotify_search():
song = sp.search(q="hello", limit=1)
features=sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

    

In [100]:
features

{'danceability': 0.578,
 'energy': 0.43,
 'key': 5,
 'loudness': -6.134,
 'mode': 0,
 'speechiness': 0.0305,
 'acousticness': 0.33,
 'instrumentalness': 0,
 'liveness': 0.0854,
 'valence': 0.288,
 'tempo': 78.991,
 'type': 'audio_features',
 'id': '62PaSfnXSMyLshYJrlTuL3',
 'uri': 'spotify:track:62PaSfnXSMyLshYJrlTuL3',
 'track_href': 'https://api.spotify.com/v1/tracks/62PaSfnXSMyLshYJrlTuL3',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/62PaSfnXSMyLshYJrlTuL3',
 'duration_ms': 295502,
 'time_signature': 4}

In [101]:
type(features)

dict

In [102]:
my_dict = sp.audio_features(song["tracks"]["items"][0]["uri"])[0] # you can provide a list of uri's

print(my_dict)
my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
#my_dict_new['name'] = [song["tracks"]["items"][0]['name']]
print(my_dict_new)

pd.DataFrame(my_dict_new)


{'danceability': 0.578, 'energy': 0.43, 'key': 5, 'loudness': -6.134, 'mode': 0, 'speechiness': 0.0305, 'acousticness': 0.33, 'instrumentalness': 0, 'liveness': 0.0854, 'valence': 0.288, 'tempo': 78.991, 'type': 'audio_features', 'id': '62PaSfnXSMyLshYJrlTuL3', 'uri': 'spotify:track:62PaSfnXSMyLshYJrlTuL3', 'track_href': 'https://api.spotify.com/v1/tracks/62PaSfnXSMyLshYJrlTuL3', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/62PaSfnXSMyLshYJrlTuL3', 'duration_ms': 295502, 'time_signature': 4}
{'danceability': [0.578], 'energy': [0.43], 'key': [5], 'loudness': [-6.134], 'mode': [0], 'speechiness': [0.0305], 'acousticness': [0.33], 'instrumentalness': [0], 'liveness': [0.0854], 'valence': [0.288], 'tempo': [78.991], 'type': ['audio_features'], 'id': ['62PaSfnXSMyLshYJrlTuL3'], 'uri': ['spotify:track:62PaSfnXSMyLshYJrlTuL3'], 'track_href': ['https://api.spotify.com/v1/tracks/62PaSfnXSMyLshYJrlTuL3'], 'analysis_url': ['https://api.spotify.com/v1/audio-analysis/62PaSfnXSMyLshYJ

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.578,0.43,5,-6.134,0,0.0305,0.33,0,0.0854,0.288,78.991,audio_features,62PaSfnXSMyLshYJrlTuL3,spotify:track:62PaSfnXSMyLshYJrlTuL3,https://api.spotify.com/v1/tracks/62PaSfnXSMyL...,https://api.spotify.com/v1/audio-analysis/62Pa...,295502,4


In [103]:
Z=pd.DataFrame(my_dict_new)

In [104]:
Z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1 non-null      float64
 1   energy            1 non-null      float64
 2   key               1 non-null      int64  
 3   loudness          1 non-null      float64
 4   mode              1 non-null      int64  
 5   speechiness       1 non-null      float64
 6   acousticness      1 non-null      float64
 7   instrumentalness  1 non-null      int64  
 8   liveness          1 non-null      float64
 9   valence           1 non-null      float64
 10  tempo             1 non-null      float64
 11  type              1 non-null      object 
 12  id                1 non-null      object 
 13  uri               1 non-null      object 
 14  track_href        1 non-null      object 
 15  analysis_url      1 non-null      object 
 16  duration_ms       1 non-null      int64  
 17  t

In [105]:
# We get numerical columns:
Z=Z._get_numeric_data()

In [106]:
scaler = StandardScaler()
scaler.fit(Z)
Z_scaled = scaler.transform(Z)
Z_scaled_df = pd.DataFrame(Z_scaled, columns = Z.columns)
display(Z.head())

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.578,0.43,5,-6.134,0,0.0305,0.33,0,0.0854,0.288,78.991,295502,4


In [107]:
clusters = kmeans2.predict(Z_scaled_df)

In [108]:
clusters

array([3])

# Create the recommender function

In [109]:
choice=["is a great choice!"," is amazing!","is cool!"]
user=""

def user_selection():
    user=input("Please enter the title of a hot song: ")
    if user in df.values:
        print(user,random.choice(choice),"You might like this one too:",random.choice(df["titles"]))
  
    else:
        song = sp.search(q=user, limit=1)
        
        
        features=sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
        
        
        my_dict_new = sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
    
        Z=pd.DataFrame(my_dict_new,index=[0])
        Z=Z._get_numeric_data()
        Z_scaled=scaler2.transform(Z)
        Z_predict_cluster=kmeans2.predict(Z_scaled)[0]
        song_to_recommend=clus_df[clus_df.cluster==Z_predict_cluster].sample(1)
        song_to_recommend_uri=song_to_recommend.uri.values
        print(song_to_recommend)
        from IPython.display import IFrame
        track_id = "7kOJsVkJXvLQPQ9osGWeKd"
        IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
        width="320",
        height="80",
        frameborder="0",
        allowtransparency="true",
        allow="encrypted-media",
        )
        
      

       
        
                                                                   
      
       


In [118]:
user_selection()

Please enter the title of a hot song: hotel california
     danceability  energy  key  loudness  mode  speechiness  acousticness  \
304         0.543   0.764    7    -7.342     1       0.0327        0.0498   

     instrumentalness  liveness  valence  ...  time_signature  cluster  \
304          0.000013    0.0626     0.43  ...               4        2   

           songs                          artists  \
304  To Her Door  Paul Kelly & The Coloured Girls   

                                     uris            type  \
304  spotify:track:2ajK6C2t7krT271fMCLbGC  audio_features   

                         id                                   uri  \
304  2ajK6C2t7krT271fMCLbGC  spotify:track:2ajK6C2t7krT271fMCLbGC   

                                            track_href  \
304  https://api.spotify.com/v1/tracks/2ajK6C2t7krT...   

                                          analysis_url  
304  https://api.spotify.com/v1/audio-analysis/2ajK...  

[1 rows x 22 columns]


In [119]:
from IPython.display import IFrame
track_id = "2ajK6C2t7krT271fMCLbGC"
IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )
